# План вебинара:

- Работаем только на CPU: преимущества и особенности.
- Работа с CatBoost: обзор библиотеки.
- Py-Boost: назначение: что это и зачем нужно.
- Установка и настройка: пошаговая инструкция.
- Обучение модели GBDT: практический пример.

## Работаем только на CPU

В этом вебинаре мы сосредоточимся на использовании центрального процессора (CPU) для выполнения всех вычислений. Такой подход имеет несколько преимуществ:

- Доступность: нет необходимости в специализированном оборудовании (GPU).
- Стабильность: меньше вероятности столкнуться с проблемами совместимости.
- Простота настройки: быстрее начать работу без сложных конфигураций.

# Работа с CatBoost

Что такое CatBoost?

CatBoost — это высокопроизводительная библиотека градиентного бустинга, разработанная компанией Яндекс. Она предназначена для:

- Эффективной работы с категориальными признаками без необходимости предварительного кодирования.
- Предотвращения переобучения благодаря использованию порядковой статистики.
- Поддержки как классификации, так и регрессии.

Преимущества CatBoost:

- Удобство: минимальная предварительная обработка данных.
- Производительность: оптимизирован для работы на CPU.
- Точность: современные алгоритмы обеспечивают высокое качество моделей.

## Установка и настройка

### Шаг 1: Установка CatBoost

Установим библиотеку с помощью pip:

pip install catboost

Если вы используете Jupyter Notebook:

!pip install catboost

### Шаг 2: Импорт необходимых библиотек

In [1]:
import pandas as pd
import numpy as np
from catboost import CatBoostClassifier, Pool
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

Обучение модели GBDT

Шаг 1: Подготовка данных

Для демонстрации возьмём датасет Titanic из open-source источника.

In [2]:
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
# Загрузка данных
data = pd.read_csv('https://raw.githubusercontent.com/datasciencedojo/datasets/master/titanic.csv')

# Просмотр первых строк
data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
# Удаляем ненужные столбцы
data = data.drop(['PassengerId', 'Name', 'Ticket', 'Cabin'], axis=1)

# Обрабатываем пропущенные значения
data['Age'].fillna(data['Age'].median(), inplace=True)
data['Embarked'].fillna(data['Embarked'].mode()[0], inplace=True)

# Преобразуем категориальные признаки
categorical_features = ['Sex', 'Embarked']
data = pd.get_dummies(data, columns=categorical_features)

# Целевая переменная и признаки
X = data.drop('Survived', axis=1)
y = data['Survived']

/var/folders/mc/nvqwydjj7b59cknzysq6s3mr0000gn/T/ipykernel_16476/2781611954.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data['Age'].fillna(data['Age'].median(), inplace=True)
/var/folders/mc/nvqwydjj7b59cknzysq6s3mr0000gn/T/ipykernel_16476/2781611954.py:6: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting value

Шаг 3: Разделение на обучающую и тестовую выборки

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

Шаг 4: Создание и обучение модели CatBoost

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [10]:
# Инициализация модели
model = CatBoostClassifier(
    iterations=1000,
    learning_rate=0.05,
    depth=10,
    eval_metric='Accuracy',
    random_seed=42,
    verbose=100
)

# Обучение модели
model.fit(
    X_train, y_train,
    eval_set=(X_test, y_test),
    use_best_model=True,
    verbose=True
)

0:	learn: 0.8244382	test: 0.7988827	best: 0.7988827 (0)	total: 1.38ms	remaining: 1.38s
1:	learn: 0.8384831	test: 0.8100559	best: 0.8100559 (1)	total: 2.2ms	remaining: 1.1s
2:	learn: 0.8384831	test: 0.8100559	best: 0.8100559 (1)	total: 3.26ms	remaining: 1.08s
3:	learn: 0.8384831	test: 0.8156425	best: 0.8156425 (3)	total: 3.84ms	remaining: 956ms
4:	learn: 0.8441011	test: 0.7877095	best: 0.8156425 (3)	total: 7.89ms	remaining: 1.57s
5:	learn: 0.8412921	test: 0.7988827	best: 0.8156425 (3)	total: 11.6ms	remaining: 1.93s
6:	learn: 0.8441011	test: 0.7988827	best: 0.8156425 (3)	total: 12.7ms	remaining: 1.79s
7:	learn: 0.8426966	test: 0.7932961	best: 0.8156425 (3)	total: 13.3ms	remaining: 1.65s
8:	learn: 0.8426966	test: 0.7932961	best: 0.8156425 (3)	total: 15.6ms	remaining: 1.72s
9:	learn: 0.8441011	test: 0.7877095	best: 0.8156425 (3)	total: 17.8ms	remaining: 1.76s
10:	learn: 0.8483146	test: 0.7932961	best: 0.8156425 (3)	total: 19.5ms	remaining: 1.76s
11:	learn: 0.8483146	test: 0.7932961	best: 0

Шаг 5: Оценка модели

In [11]:
# Предсказание
y_pred = model.predict(X_test)

# Оценка точности
accuracy = accuracy_score(y_test, y_pred)
print(f"Точность модели: {accuracy:.4f}")

Точность модели: 0.8492


# Заключение

В ходе данного вебинара мы:

Познакомились с библиотекой CatBoost и её преимуществами.
Узнали о назначении Py-Boost и его роли в экосистеме Python.
Прошли через процесс установки и настройки CatBoost.
На практическом примере обучили модель GBDT на данных о пассажирах Титаника.
Оценили качество модели и убедились в эффективности подхода.


## Дополнительные задания

Чтобы закрепить материал, вы можете:

Попробовать изменить гиперпараметры модели и посмотреть, как это повлияет на точность.
Использовать другой датасет для обучения модели.
Изучить методы обработки категориальных данных в CatBoost без предварительного кодирования.